In [ ]:
import tensorflow as tf
print(f'{tf.__version__}')

2.3.0


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,Activation
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
mnist=tf.keras.datasets.mnist
(X_train,y_train),(X_test,y_test)=mnist.load_data()

In [ ]:
X_train=tf.cast(np.reshape(X_train,(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)),tf.float64)

In [ ]:
X_test

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [ ]:
X_test = tf.cast(np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)), tf.float64)


In [ ]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [ ]:
y_train=tf.keras.utils.to_categorical(y_train)
y_test=tf.keras.utils.to_categorical(y_test)

In [ ]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [ ]:
model = tf.keras.models.Sequential()
model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28,28,1)))
model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_uniform'))
model.add(Flatten())
model.add(Dense(20))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 20)                737300    
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
Total params: 756,326
Trainable params: 756,326
Non-trainable params: 0
__________________________________________________

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 2s 5ms/step - loss: 14.9029 - accuracy: 0.8924
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.2022 - accuracy: 0.9563
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1284 - accuracy: 0.9673
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0968 - accuracy: 0.9741
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1015 - accuracy: 0.9740


In [ ]:
!pip install -q git+https://github.com/keras-team/keras-tuner

In [ ]:
import kerastuner
from kerastuner.tuners import RandomSearch

In [ ]:
def model_fn(hp):
    LR = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])
    DROPOUT_RATE = hp.Float('dropout_rate', 0.0, 0.5, 5)
    NUM_DIMS = hp.Int('num_dims',  8, 32, 8)
    NUM_LAYERS = hp.Int('num_layers', 1, 3)
    L2_NUM_FILTERS = hp.Int('l2_num_filters', 8, 64, 8)
    L1_NUM_FILTERS = hp.Int('l1_num_filters', 8, 64, 8)
    model = tf.keras.models.Sequential()
    model.add(Conv2D(L1_NUM_FILTERS, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28,28,1)))
    model.add(Conv2D(L2_NUM_FILTERS, (3,3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Flatten())
    for _ in range(NUM_LAYERS):
        model.add(Dense(NUM_DIMS))
        model.add(Dropout(DROPOUT_RATE))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    model_fn,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=3,
    directory='temp_dir')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0005, 0.0001], 'ordered': True}
dropout_rate (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 5.0, 'sampling': None}
num_dims (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 32, 'step': 8, 'sampling': None}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
l2_num_filters (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 64, 'step': 8, 'sampling': None}
l1_num_filters (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 64, 'step': 8, 'sampling': None}


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 01m 22s]
val_accuracy: 0.9616999824841818

Best val_accuracy So Far: 0.9649666547775269
Total elapsed time: 00h 05m 57s
INFO:tensorflow:Oracle triggered exit


In [ ]:

models = tuner.get_best_models(num_models=3)

In [ ]:
models

In [ ]:
tuner.results_summary()

Results summary
Results in temp_dir/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.0005
dropout_rate: 0.0
num_dims: 32
num_layers: 2
l2_num_filters: 48
l1_num_filters: 8
Score: 0.9649666547775269
Trial summary
Hyperparameters:
learning_rate: 0.0005
dropout_rate: 0.0
num_dims: 8
num_layers: 2
l2_num_filters: 32
l1_num_filters: 32
Score: 0.9616999824841818
Trial summary
Hyperparameters:
learning_rate: 0.0005
dropout_rate: 0.0
num_dims: 16
num_layers: 3
l2_num_filters: 64
l1_num_filters: 40
Score: 0.9575000007947286
Trial summary
Hyperparameters:
learning_rate: 0.0001
dropout_rate: 0.0
num_dims: 24
num_layers: 2
l2_num_filters: 56
l1_num_filters: 16
Score: 0.9556000034014384
